In [5]:
import gym
import numpy as np
import time
import random
import operator
import math
from IPython.display import clear_output
from gym import wrappers
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [6]:
env = gym.make("Acrobot-v1")
#env = wrappers.Monitor(env, '/tmp/cartpole-experiment-5')
env.reset();

In [7]:
env.step(1)

(array([ 0.99981321, -0.01932756,  0.99958986, -0.02863761,  0.07393875,
        -0.0093266 ]), -1.0, False, {})

In [4]:
for i in range(100):
    env.step(random.randint(0,1))
    env.render()
env.reset()

array([ 0.99886988, -0.0475286 ,  0.99936722, -0.03556922,  0.00476106,
       -0.03119897])

In [8]:
pset = gp.PrimitiveSet("main", 6)
def if_then_else(input, output1, output2):
    return output1 if input else output2

def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    


pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)


In [9]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [10]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

In [11]:
def sample_reward(env, func, t_max=200):
    s = env.reset()
    total_reward = 0
    max_pos = -1
    #self.rewardx = (-np.cos(s[0]) - np.cos(s[1] + s[0]))
    for _ in range(t_max):
        next_action = func(*s)
        if next_action<=-1:
            next_action = -1
        elif next_action>-1 and next_action<1:
            next_action = 0
        else:
            next_action = 1

        s, reward, is_done, _ = env.step(next_action)
        rewardx = (-np.cos(s[0]) - np.cos(s[1] + s[0]))
        
        if rewardx>max_pos:
            max_pos = rewardx 
        if is_done:
            return max_pos
    return max_pos

In [12]:
def evalSymbReg(individual, n_times=10):
    
    func = toolbox.compile(expr=individual)
    rewards = [sample_reward(env, func) for _ in range(n_times)]
    
    return float(np.mean(rewards)), 

In [13]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [14]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [15]:
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


   	      	                 fitness                 	              size             
   	      	-----------------------------------------	-------------------------------
gen	nevals	avg    	max      	min	std      	avg    	max	min	std    
0  	300   	-0.9238	-0.851067	-1 	0.0417243	20.6667	116	2  	26.7199
1  	171   	-0.878199	-0.851065	-1 	0.0410447	34.53  	109	1  	33.2313
2  	161   	-0.869955	-0.851064	-1 	0.0419448	37.71  	142	1  	35.2439
3  	160   	-0.869562	-0.851064	-1 	0.0412487	33.8633	146	3  	37.8254
4  	179   	-0.87274 	-0.851064	-1 	0.044796 	31.1033	156	3  	35.3994
5  	156   	-0.870928	-0.851064	-1 	0.0430574	28.6   	156	1  	33.5577
6  	159   	-0.87682 	-0.851063	-1 	0.048047 	19.32  	115	1  	27.0669
7  	165   	-0.879025	-0.851063	-1 	0.0459705	7.76   	100	1  	9.43623
8  	176   	-0.881698	-0.851063	-1 	0.0492509	8.06   	94 	1  	6.72778
9  	159   	-0.879009	-0.851063	-1 	0.050314 	9.32333	28 	3  	5.61831
10 	155   	-0.872459	-0.851063	-1 	0.0435839	13.6633	35 	3  	7.09718
11 	17

In [15]:
best_func = gp.compile(hof[0], pset)

In [16]:
evalSymbReg(hof[0])

(0.9,)

In [ ]:
for i in range(100):
    s = env.reset()
    total_reward = 0
    for _ in range(500):
        next_action = best_func(*s)
        next_action = 0 if next_action<=0 else 1
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        env.render()
        if is_done:
            break


TypeError: must be str, not bytes